<h1 style='color: green; font-size: 36px; font-weight: bold;'>Exemplo de uso do Tribuo</h1>

# <font color='red' style='font-size: 30px;'>Baixar a lib do Tribuo</font>
<hr style='border: 2px solid red;'>

In [1]:
%%loadFromPOM
<dependency>
<groupId>org.tribuo</groupId>
<artifactId>tribuo-all</artifactId>
<version>4.3.1</version>
<type>pom</type>
</dependency>

# <font color='black' style='font-size: 20px;'>Imports para manipular arquivos</font>
<hr style='border: 2px solid black;'>

In [2]:
import java.nio.file.Path;
import java.nio.file.Paths;

# <font color='black' style='font-size: 20px;'>Imports para o uso do Tribuo</font>
<hr style='border: 2px solid black;'>

In [3]:
import org.tribuo.*;
import org.tribuo.data.csv.CSVLoader;
import org.tribuo.datasource.ListDataSource;
import org.tribuo.evaluation.TrainTestSplitter;
import org.tribuo.math.optimisers.*;
import org.tribuo.regression.*;
import org.tribuo.regression.evaluation.*;
import org.tribuo.regression.sgd.RegressionObjective;
import org.tribuo.regression.sgd.linear.LinearSGDTrainer;
import org.tribuo.regression.sgd.objectives.SquaredLoss;
import org.tribuo.regression.rtree.CARTRegressionTrainer;
import org.tribuo.regression.xgboost.XGBoostRegressionTrainer;
import org.tribuo.util.Util;

# <font color='black' style='font-size: 20px;'>Criar instância do regressionFactory e csv Loader para ler o arquivo CSV</font>
<hr style='border: 2px solid black;'>

In [4]:
var regressionFactory = new RegressionFactory();
var csvLoader = new CSVLoader<>(';',regressionFactory);

# <font color='black' style='font-size: 20px;'>Carregar dados de um arquivo CSV, divide-os em conjuntos de treino e teste, e prepara esses dados para treinar e avaliar um modelo de regressão.</font>
<hr style='border: 2px solid black;'>

In [5]:
var wineSource = csvLoader.loadDataSource(Paths.get("tribuo_alura/winequality-red.csv"),"quality");
var splitter = new TrainTestSplitter<>(wineSource, 0.7f, 0L);
Dataset<Regressor> trainData = new MutableDataset<>(splitter.getTrain());
Dataset<Regressor> evalData = new MutableDataset<>(splitter.getTest());

<p color='black' style='font-size: 20px;'>Realizar a extração e exibição de informações sobre as dimensões do primeiro exemplo de saída de um modelo de regressão. O código exibe o número de dimensões, o nome e o valor da primeira dimensão, além de acessar e imprimir tuplas específicas das dimensões com base no nome e no índice fornecidos.</p>

In [6]:
Regressor r = trainData.getExample(0).getOutput();
System.out.println("Num dimensions = " + r.size());

String[] dimNames = r.getNames();
System.out.println("Dimension name: " + dimNames[0]);

double[] regressedValues = r.getValues();
System.out.println("Dimension value: " + regressedValues[0]);

// getDimension(String) returns an Optional<DimensionTuple>
Regressor.DimensionTuple tuple = r.getDimension("DIM-0").get();
System.out.println("Tuple = [" + tuple +"]");

// getDimension(int) throws IndexOutOfBoundsException if you give it a negative index
// or one greater than or equal to r.size()
Regressor.DimensionTuple tupleI = r.getDimension(0);
System.out.println("Regressor[0] = " + tupleI);

Num dimensions = 1
Dimension name: DIM-0
Dimension value: 6.0
Tuple = [DIM-0=6.0]
Regressor[0] = DIM-0=6.0


<p style='font-size: 20px;'>
Realizar o treinamento de um modelo de regressão e avaliar seu desempenho no conjunto de dados de treinamento. </p>

In [7]:
public Model<Regressor> train(String name, Trainer<Regressor> trainer, Dataset<Regressor> trainData) {
    // Train the model
    var startTime = System.currentTimeMillis();
    Model<Regressor> model = trainer.train(trainData);
    var endTime = System.currentTimeMillis();
    System.out.println("Training " + name + " took " + Util.formatDuration(startTime,endTime));
    // Evaluate the model on the training data
    // This is a useful debugging tool to check the model actually learned something
    RegressionEvaluator eval = new RegressionEvaluator();
    var evaluation = eval.evaluate(model,trainData);
    // We create a dimension here to aid pulling out the appropriate statistics.
    // You can also produce the String directly by calling "evaluation.toString()"
    var dimension = new Regressor("DIM-0",Double.NaN);
    System.out.printf("Evaluation (train):%n  RMSE %f%n  MAE %f%n  R^2 %f%n",
            evaluation.rmse(dimension), evaluation.mae(dimension), evaluation.r2(dimension));
    return model;
}

<p style='font-size: 20px;'>Realizar a avaliação de um modelo de regressão utilizando dados de teste.</p>

In [8]:
public void evaluate(Model<Regressor> model, Dataset<Regressor> testData) {
    // Evaluate the model on the test data
    RegressionEvaluator eval = new RegressionEvaluator();
    var evaluation = eval.evaluate(model,testData);
    // We create a dimension here to aid pulling out the appropriate statistics.
    // You can also produce the String directly by calling "evaluation.toString()"
    var dimension = new Regressor("DIM-0",Double.NaN);
    System.out.printf("Evaluation (test):%n  RMSE %f%n  MAE %f%n  R^2 %f%n",
            evaluation.rmse(dimension), evaluation.mae(dimension), evaluation.r2(dimension));
}

<p style='font-size: 20px;'>Configurar diferentes algoritmos de treinamento para modelos de regressão linear.</p>

In [9]:
var lrsgd = new LinearSGDTrainer(
    new SquaredLoss(), // loss function
    SGD.getLinearDecaySGD(0.01), // gradient descent algorithm
    10,                // number of training epochs
    trainData.size()/4,// logging interval
    1,                 // minibatch size
    1L                 // RNG seed
);
var lrada = new LinearSGDTrainer(
    new SquaredLoss(),
    new AdaGrad(0.01),
    10,
    trainData.size()/4,
    1,
    1L 
);
var cart = new CARTRegressionTrainer(6);
var xgb = new XGBoostRegressionTrainer(50);

<p style='font-size: 20px;'>Treinar um modelo de regressão linear utilizando o treinador lrsgd e o conjunto de dados de treinamento trainData.</p>

In [10]:
var lrsgdModel = train("Linear Regression (SGD)",lrsgd,trainData);


Training Linear Regression (SGD) took (00:00:00:056)
Evaluation (train):
  RMSE 0.979522
  MAE 0.741870
  R^2 -0.471611


In [11]:
evaluate(lrsgdModel,evalData);

Evaluation (test):
  RMSE 0.967450
  MAE 0.720619
  R^2 -0.439255


In [12]:
var lradaModel = train("Linear Regression (AdaGrad)",lrada,trainData);
evaluate(lradaModel,evalData);

Training Linear Regression (AdaGrad) took (00:00:00:032)
Evaluation (train):
  RMSE 0.735311
  MAE 0.575096
  R^2 0.170709
Evaluation (test):
  RMSE 0.737994
  MAE 0.585709
  R^2 0.162497


In [20]:
var cartModel = train("CART",cart,trainData);
evaluate(cartModel,evalData);

Training CART took (00:00:00:075)
Evaluation (train):
  RMSE 0.544516
  MAE 0.405062
  R^2 0.545236
Evaluation (test):
  RMSE 0.658722
  MAE 0.494395
  R^2 0.332754


RMSE (Erro Quadrático Médio)

RMSE 0.544516: Indica a média dos erros ao quadrado entre as previsões do modelo e os valores observados nos dados de treinamento. Um valor menor de RMSE indica um melhor ajuste. Esse valor reflete o quão distantes estão as previsões do modelo dos valores reais.
MAE (Erro Absoluto Médio)

MAE 0.405062: Indica a média dos valores absolutos dos erros entre as previsões do modelo e os valores reais nos dados de treinamento. Assim como o RMSE, valores menores de MAE indicam um melhor ajuste. A diferença é que o MAE não penaliza grandes erros tanto quanto o RMSE.
R² (Coeficiente de Determinação)

R^2 0.545236: Indica a proporção da variância nos dados de treinamento que é explicada pelo modelo. O valor de R² varia de 0 a 1 (podendo ser negativo), onde 1 indica um modelo perfeito. Um valor de 0.545236 significa que cerca de 54,5% da variabilidade dos dados de treinamento é explicada pelo modelo. Valores mais próximos de 1 indicam um bom ajuste, enquanto valores negativos indicam que o modelo é pior do que simplesmente usar a média dos dados.


O modelo CART treinado conseguiu capturar alguma informação útil dos dados (R² acima de 0,5 nos dados de treinamento), mas seu desempenho caiu consideravelmente nos dados de teste. Isso indica que o modelo pode não estar generalizando bem para novos dados. Para melhorar, você pode considerar:

Ajustar hiperparâmetros do modelo.
Aumentar o número de amostras no conjunto de dados de treinamento.
Usar técnicas de regularização para evitar overfitting.
Testar outros modelos, como Random Forest ou XGBoost.